In [1]:
from model import Model
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import Trainer,TrainingArguments, DefaultDataCollator
import torch
import transformers
from torch.utils.tensorboard import SummaryWriter

In [2]:
emotation_ds = load_dataset("dair-ai/emotion", "split")

In [3]:
emotation_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
gpt2_tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

In [5]:
tokenizer = gpt2_tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [6]:
model = Model().to("cuda")

In [7]:
def preprocess(examples):
    model_inputs = tokenizer(examples["text"], max_length=768,padding=True,truncation=True)
    return model_inputs

In [8]:
emotation_ds = emotation_ds.map(preprocess,batched=True,remove_columns="text")

In [9]:
training_args = TrainingArguments(
    output_dir="my_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    remove_unused_columns=True,
    logging_dir = 'log',
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    save_steps=20,
    logging_steps=20,
    learning_rate=0.001,
    num_train_epochs=1,
    max_steps=100,
    eval_steps=10,
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=emotation_ds["train"],
    eval_dataset=emotation_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=transformers.integrations.TensorBoardCallback(SummaryWriter("./tensorboard")),
)

trainer.train()
trainer.save_model("./")

max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/100 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 